<img src="media/nuplan.png" alt="Drawing" style="width: 700px;"/>

# Introduction to nuPlan <a name="introduction"></a>


Welcome to nuPlan! This notebook will explore the nuPlan simulation framework, training platform as well as the nuBoard metrics/scenarios visualization dashboard.

## Contents

1. [Introduction to nuPlan](#introduction)
2. [Training an ML planner](#training)
3. [Simulating a planner](#simulation)
4. [Visualizing metrics and scenarios](#dashboard)

## What is nuPlan

nuPlan is the world’s first closed-loop ML-based planning benchmark for autonomous driving.

It provides a high quality dataset with 1500h of human driving data from 4 cities across the US and Asia with widely varying traffic patterns (Boston, Pittsburgh, Las Vegas and Singapore). In addition, it provides a closed-loop simulation framework with reactive agents, a training platform as well as a large set of both general and scenario-specific planning metrics.

## Training & simulation framework

The nuPlan training and simulation framework aims to:
* create a simulation pipeline to evaluate a planner on large dataset with various scenarios
* score planner performance with common and scenario-dependent metrics
* compare planners based on measured metrics and provide intuitive visualizations
* train planners with the provided framework to allow quick implementation and iteration
* support closed-loop simulation and training

<br />

Framework overview

<img src="media/framework.png" alt="Drawing" style="width: 700px;"/>

## Scenarios in nuPlan

nuPlan aims to capture challenging yet representative scenarios from real-world encounters. This enables the benchmarking of planning systems both in expert imitation (open-loop) and reactive planning (closed-loop) settings.

These scenarios includes:
* highly interactive scenes with traffic participants (e.g. tailgating, high-velocity overtakes, double parked cars, jaywalking)
* various ego behaviors (e.g. vehicle following, yielding, lane merging) and dynamics (e.g. mixed speed profiles, abrupt braking, speed bumps, high jerk maneuvers)
* scene layouts of varied complexity (e.g. pudos, traffic/stop controlled intersections, unprotected turns) and temporary zones (e.g. construction areas)

The dataset is automatically tagged with scenario labels based on certain primitive attributes. These scenario tags can then be used to extract representative metrics for the planner's evaluation.

<br />

Available scenarios (under development)

<img src="media/scenarios.png" alt="Drawing" style="width: 700px;"/>

To be able to access all resources within this notebook, make sure Jupyter is launched at the root of this repo. Then the path inside the notebook looks like: `/notebook/nuplan/..`.

In [ ]:
# Increase notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Useful imports
import os
from pathlib import Path
import tempfile

import hydra

In [ ]:
# Set downloaded model path from previous step to be used in simulation
MODEL_PATH = f'/data/exp/{os.getlogin()}/nuplan_models/raster_model_8h.ckpt'

# 1) Training an ML planner <a name="training"></a>

## Imitation learning
In the following section we will train an ML planning policy with the aim estimate the ego's future trajectory and control the vehicle.

The policy is learned through imitation learning, a supervised learning approach in which - in the context of autonomous driving - the behavior of an expert human driver is used as a target signal to supervise the model.

## Model features & targets
A planning policy consumes a set of episodic observations and encodes them through a deep neural network to regress a future trajectory.

The observations can be historic or present ego and agent poses as well as static/dynamic map information across different map layers.<br />
These signals can be encoded through various representations, such as raster or vector format for the map signal, each with their pros and cons for each model flavor.

Using these input features the model predicts a discretized future trajectory across a fixed time horizon.<br />
The trajectory consists of a set of discrete future states (position, heading and velocity) sampled at fixed intervals which express the likelihood of the vehicle being at that state in the future.<br />
For example, a predicted trajectory may consist of 10 future poses sampled at intervals of 0.5s across a 5s horizon.

## Learning objectives
The policy is trained to maximize a set of aggregated objectives such as imitation, collision avoidance, traffic rule violation etc.<br />
Imitation is the core training objective which indicates how close the predicted trajectory is to the expert ground truth and penalizes model predictions that deviate in space and time from the demonstration.

## Training parameters

The following parameter categories define the training protocol which includes the model, database, metrics, objectives etc.

A working example composition of these parameters can be found in the next section.

---

### ML models

Change the training model with `model=X` where `X` is a config yaml defined in the table below. 

| Model | Description | Config |
| --- | --- | --- |
| CNN raster model | Raster-based model that uses a CNN backbone to encode ego, agent and map information as raster layers<br />Any (pretrained) backbone from the TIMM library can be used (e.g. ResNet50, EfficientNetB3) | `raster_model` |
| LaneGCN vector model | Vector-based model that uses a series of MLPs to encode ego and agent signals, a lane graph to encode vector-map elements and a fusion network to capture lane & agent intra/inter-interactions through attention layers<br />Implementation of LaneGCN paper ("Learning Lane Graph Representations for Motion Forecasting") | `vector_model` |
| Toy vector model | Toy vector-based model that consumes ego, agent and lane signals through a series of MLPs | `simple_vector_model` |

<br />

### Training objectives

Change the training objectives with `objective=[X, ...]` where `X` is a config yaml defined in the table below. 

| Objective | Description | Config |
| --- | --- | --- |
| Imitation objective | Penalizes the predicted trajectory that deviates from the expert demonstration | `imitation_objective` |
| Collision objective | Penalizes the predicted trajectory that collides with other agents | Coming soon... |
| Violation objective | Penalizes the predicted trajectory which is outside of the drivable area | Coming soon... |

<br />

### Training metrics

Change the training objectives with `training_metric=[X, ...]` where `X` is a config yaml defined in the table below. 

| Metric | Description | Config |
| --- | --- | --- |
| Average displacement error | RMSE translation error across full predicted trajectory | `avg_displacement_error` |
| Average heading error | RMSE heading error across full predicted trajectory | `avg_heading_error` |
| Final displacement error | L2 error of predicted trajectory's final pose translation | `final_displacement_error` |
| Final heading error | L2 error of predicted trajectory's final pose heading | `final_heading_error` |

<br />

### Scenario Builders

Change the scenario_builder with `scenario_builder=<X>` where `X` is a config yaml defined in the table below. 

| Database | Description | Config |
| --- | --- | --- |
| nuplan_v0.2 | 102h of driving in LV with scenario tags (89 autolabeled logs) | `nuplan` |
| nuplan_v0.2_mini | 2h of driving in LV with scenario tags (4 autolabeled logs) | `nuplan_mini`
<br />

### Scenario filters

Change the type of scenario_filter with `scenario_builder/<database>/scenario_filter=<X>` where `X` is a config yaml defined in `script/common/scenario_builder/<database>/scenario_filter`. <br />
Change a parameter of a scenario type with `scenario_builder.<database>.<X>` where `X` is a parameter in the scenario filter. <br />
For nuPlan, the default scenario filters have the followiing parameters defined in the table below. 

| Filter | Description | Config |
| --- | --- | --- |
| log_names | List of logs name to extract scenarios from | `scenario_builder.nuplan.scenario_filter.log_names` |
| log_labels | Log tags used to filter logs | `scenario_builder.nuplan.scenario_filter.log_labels` |
| max_scenarios_per_log | Maximum number of scenarios extracted per log | `scenario_builder.nuplan.scenario_filter.max_scenarios_per_log` |
| scenario_types | List of scenario types to be used | `scenario_builder.nuplan.scenario_filter.scenario_types` |
| map_name | Filter out scenarios on a specific map | `scenario_builder.nuplan.scenario_filter.map_name` |
| shuffle | Shuffle selected scenarios | `scenario_builder.nuplan.scenario_filter.shuffle` |
| limit_scenarios_per_type | How many scenarios per type to keep (float = fraction, int = num) | `scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type` |
| subsample_ratio | Subsample a scenario relative to the database frequency (e.g. nominal freq @ 20Hz means 400 examples for 20s -> 2.5% subsample_ratio means 10 examples for 20s) | `scenario_builder.nuplan.scenario_filter.subsample_ratio` |
| flatten_scenarios | Flatten scenarios (e.g. 10x 20-samples scenarios to 200x 1-sample scenarios) | `scenario_builder.nuplan.scenario_filter.flatten_scenarios` |
| remove_invalid_goals | Remove scenarios with invalid goals (e.g on an intersection, exceeded max dist) | `scenario_builder.nuplan.scenario_filter.remove_invalid_goals` |
| limit_total_scenarios | How many total scenarios to keep (float = fraction, int = num) | `scenario_builder.nuplan.scenario_filter.limit_total_scenarios` |

## Prepare the training config

In [ ]:
# Location of path with all training configs
CONFIG_PATH = '../script/config/training'
CONFIG_NAME = 'default_training'

# Create a temporary directory to store the cache and experiment artifacts
save_dir = tempfile.mkdtemp()
experiment_name = 'open_loop_experiment'
log_dir = str(Path(save_dir) / experiment_name)

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'group={save_dir}',
    f'cache_dir={save_dir}/cache',
    f'experiment_name={experiment_name}',
    'py_func=train',
    '+training=training_raster_model',  # raster model that consumes ego, agents and map raster layers and regresses the ego's trajectory
    'scenario_builder=nuplan_mini',  # use nuplan mini database (2h of 4 autolabeled logs in LV)
    'splitter=nuplan',
    'scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=0.1',  # sample 10% of the dataset
    'scenario_builder.nuplan.scenario_filter.subsample_ratio=0.05',  # subsample 20s scenarios from 400 (20Hz) to 20 (1Hz) lidarpcs
    'lightning.trainer.params.max_epochs=3',
    'lightning.trainer.params.accelerator=ddp_spawn',  # ddp is not allowed in interactive environment, using ddp_spawn instead
    'data_loader.params.batch_size=2',
    'data_loader.params.num_workers=0',
])

## Launch tensorboard for visualizing training artifacts

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

## Launch training (within the notebook)

In [ ]:
from nuplan.planning.script.run_training import main as main_train

# Run the training loop, optionally inspect training artifacts through tensorboard (above cell)
main_train(cfg)

## Launch training (command line - alternative)

A training experiment with the above same parameters can be launched alternatively with:
```
$ python nuplan_devkit/nuplan/planning/script/run_training.py \
    group=/data/exp/$USER/nuplan \
    cache_dir=/data/exp/$USER/cache \
    experiment_name=open_loop_experiment \
    py_func=train \
    +training=training_raster_model \
    scenario_builder=nuplan_mini \
    splitter=nuplan \
    scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=1.0 \
    scenario_builder.nuplan.scenario_filter.subsample_ratio=0.1 \
    lightning.trainer.params.max_epochs=5 \
    data_loader.params.batch_size=8 \
    data_loader.params.num_workers=8
```

# 2) Simulating a planner <a name="simulation"></a>

## Open-loop simulation
Open-loop simulation aims to evaluate the policy's capabilities to imitate the expert driver's behavior.<br />
This is essentially done through log replay as the policy's predictions do not affect the state of the simulation.

As the policy is not in full control of the vehicle, this type of simulation can only provide a high-level performance overview.

## Closed-loop simulation
Conversely, in closed-loop simulation the policy's actions alter the state of the simulation which tries to closely approximate the real-world system.

The simulation's feedback loop enables a more in-depth evaluation of the policy as compounding errors can cause future observations to significantly diverge from the ground truth.<br />
This is important in measuring distribution shifts introduced due to lack of variance in training examples through pure imitation learning.

Closed-loop simulation is further divided into two categories:
* ego closed-loop simulation with agents replayed from log (open-loop, non reactive)
* ego closed-loop simulation with agents controlled by a rule-based or learned policy (closed-loop, reactive)

## Measuring success
Measuring the success of a planning task and comparing various planning policies is a complicated effort that involves defining metrics across different vertical dimensions and scenario categories.<br />
These metrics include indicators such as vehicle dynamics, traffic rule violations, expert imitation, navigation success etc.<br />
Overall, they aim to capture the policy's ability to control the autonomous vehicle safely yet efficiently without compromising the passenger's comfort.

## Simulation parameters

### Planners

Change the planner model with `planner=X` where `X` is a config yaml defined in the table below. 

| Planner | Description | Config |
| --- | --- | --- |
| Simple Planner | Naive planner that only plans a straight path | `simple_planner` |
| Borromean Rings Planner | Internal rule-based planner | `borromean_rings` |
| ML Planner (raster model) | Learning-based planner trained using the nuPlan training framework (see previous section) | `ml_planner`<br />`planner.checkpoint_path="<path_to_model_ckpt>"`<br />`model="<selected model>"`<br />`planner.model_config=\${model}` |
| ML Planner (vector model) | Learning-based planner trained using the nuPlan training framework (see previous section) | Coming soon... |

<br />


### Evaluation metrics

Select evaluation metrics with `selected_simulation_metrics=[X, ...]` where `X` is a config yaml defined in the table below (by default all metrics are selected).

| Metric | Description | Config |
| --- | --- | --- |
| Distance to goal | Distance to target goal pose | `ego_distance_to_goal_statistics` |
| Angular velocity | Ego yaw rate | `ego_yaw_rate_statistics` |
| Linear acceleration | Ego combined acceleration | `ego_acceleration_statistics` |
| Angular acceleration | Ego yaw acceleration | `ego_yaw_acceleration_statistics` |
| Longitudinal acceleration | Longitudinal component of the ego acceleration | `ego_lon_acceleration_statistics` |
| Lateral acceleration | Lateral component of the ego acceleration | `ego_lat_acceleration_statistics` |
| Linear jerk | Ego combined jerk | `ego_jerk_statistics` |
| Longitudinal jerk | Longitudinal component of the ego jerk | `ego_lon_jerk_statistics` |
| Lateral jerk | Lateral component of the ego jerk | `ego_lat_jerk_statistics` |
| Minimum distance to agent | Minimum distance of ego to a track | `ego_min_distance_to_track_statistics` |
| Time gap to leading agent | Time taken for the ego to close the gap if the lead agent were to abruptly stop | `time_gap_to_lead_agent` |
| Time gap to trailing agent | Time taken for the ego to close the gap if the trail agent were to abruptly stop | `time_gap_to_rear_agent` |
| Comfort | Aggregation of min/max velocity, acceleration & jerk across all components | `ego_is_comfortable_statistics` |

<br />

Scenario building and scenario filtering configs are the same for training and simulation (see the training section above).

## Prepare the simulation config

In [ ]:
# Location of path with all simulation configs
CONFIG_PATH = '../script/config/simulation'
CONFIG_NAME = 'default_simulation'

# Create a temporary directory to store the simulation artifacts
SAVE_DIR = tempfile.mkdtemp()

# Select the planner and simulation challenge
PLANNER = 'simple_planner'  # [simple_planner, borromean_rings, ml_planner]
CHALLENGE = 'challenge_1_open_loop_boxes'  # [challenge_1_open_loop_boxes, challenge_3_closed_loop_nonreactive_agents, challenge_4_closed_loop_reactive_agents]
DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database (2h of 4 autolabeled logs in LV)
    'scenario_builder/nuplan/scenario_filter=one_continuous_log',  # simulate only one log
    'scenario_builder.nuplan.scenario_filter.log_names="[2021.05.27.14.27.08_g1p-veh-2035]"',  # select one log from test set of nuplan mini
    'scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=50',  # use 50 scenarios per type
    'scenario_builder.nuplan.scenario_filter.subsample_ratio=0.05',  # subsample 20s scenario from 20Hz to 1Hz
]

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'group={SAVE_DIR}',
    f'planner={PLANNER}',
    f'+simulation={CHALLENGE}',
    *DATASET_PARAMS,
])

## Launch simulation (within the notebook)

In [ ]:
from nuplan.planning.script.run_simulation import main as main_simulation

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg)

# Fetch the filesystem location of the simulation results file for visualization in nuBoard (next section)
results_dir = list(list(Path(SAVE_DIR).iterdir())[0].iterdir())[0]  # get the child dir 2 levels in
nuboard_file_1 = [str(file) for file in results_dir.iterdir() if file.is_file() and file.suffix == '.nuboard'][0]

## Launch simulation (command line - alternative)

A simulation experiment can be launched alternatively with:
```
$ python nuplan_devkit/nuplan/planning/script/run_simulation.py \
    +simulation=challenge_1_open_loop_boxes \
    planner=simple_planner \
    scenario_builder=nuplan_mini \
    scenario_builder/nuplan/scenario_filter=one_continuous_log \
    scenario_builder.nuplan.scenario_filter.log_names="[2021.05.27.14.27.08_g1p-veh-2035]" \
    scenario_builder.nuplan.scenario_filter.limit_scenarios_per_type=50 \
    scenario_builder.nuplan.scenario_filter.subsample_ratio=0.1
```

The simple toy planner can be replaced by the ML planner model trained in the previous section.

## Simulate a pretrained ML planner for comparison

Using the same simulation settings as before, we can simulate a pretrained ML planner and compare the two.

Make sure to download the model using the instructions at the top.

In [ ]:
# Create a temporary directory to store the simulation artifacts
SAVE_DIR = tempfile.mkdtemp()

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'group={SAVE_DIR}',
    'planner=ml_planner',
    'model=raster_model',
    'planner.model_config=${model}',
    f'planner.ml_planner.checkpoint_path={MODEL_PATH}',  # this path can be replaced by the checkpoint of the model trained in the previous section
    f'+simulation={CHALLENGE}',
    *DATASET_PARAMS,
])

# Run the simulation loop
main_simulation(cfg)

# Fetch the filesystem location of the simulation results file for visualization in nuBoard (next section)
results_dir = list(list(Path(SAVE_DIR).iterdir())[0].iterdir())[0]  # get the child dir 2 levels in
nuboard_file_2 = [str(file) for file in results_dir.iterdir() if file.is_file() and file.suffix == '.nuboard'][0]

# 3) Visualizing metrics and scenarios <a name="dashboard"></a>

## nuBoard summary

Having trained and simulated planners across various scenarios and driving behaviors, it's time to evaluate them:
* quantitatively, through common and scenario dependent metrics
* qualitatively, through visualization of scenario progression

### nuBoard tabs
To achieve that, nuBoard has 3 core evaluation tabs:
1. Overview - Scalar metrics summary of common and scenario metrics across the following categories:
    * Ego dynamics
    * Traffic violations
    * Expert imitation
    * Planning & navigation
    * Scenario performance
2. Histograms - Histograms over metric statistics for more a granular peek inside each metric focusing on:
    * Metric statistics (e.g. min, max, p90)
3. Scenarios - Low-level scenario visualizations:
    * Time-series progression of a specific metric across a scenario
    * Top-down visualization of the scenario across time for comparing predicted vs. expert trajectories

In addition, there is a main configuration tab for selecting different simulation files for comparing planners/experiments.

<br />

**NOTE**: nuBoard is under heavy developement, overall functionality and aesthetics do not represent the final product!

## Prepare the nuBoard config

In [ ]:
# Location of path with all nuBoard configs
CONFIG_PATH = '../script/config/nuboard'
CONFIG_NAME = 'default_nuboard'

simulation_files = [
    nuboard_file_1,
    nuboard_file_2,  # comment out if the pretrained model simulation hasn't run
]

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    'scenario_builder=nuplan_mini',  # set the database (same as simulation) used to fetch data for visualization
    f'simulation_path={simulation_files}',  # nuboard file path, if left empty the user can open the file inside nuBoard
])

## Launch nuBoard (embedded within the notebook - recommended)

In [ ]:
from bokeh.io import show, output_notebook
from nuplan.planning.script.run_nuboard import initialize_nuboard
import os

# Pass the CSS resources to the notebook
cfg.bokeh.resource_prefix = '/notebooks/nuplan/planning/metrics/board/'

# Run the nuBoard
output_notebook()
nuboard = initialize_nuboard(cfg)
show(nuboard.main_page)

## Launch nuBoard (open in new tab - alternative)

In [ ]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)

## Launch nuBoard (command line - alternative)

nuBoard can be launched alternatively with:
```
$ python ~/nuplan_devkit/nuplan/planning/script/run_nuboard.py
```

Simulation files (.nuboard) can be selected under the configuration tab.